# **5XSM0 - MRI for brain**
In this lab we will load and preprocess functional MRI (fMRI) images. Subsequently, the corrected images can be used to extract default-mode network (DMN) activation.

Goals for this lab are:
  - Correct for EPI distortions (using FSL's topup)
  - Correct for subject motion (using FSL's mcflirt)
  - Correct for nuisance (using FSL's reg_filt)
  - Extract DMN activity



First, we must set up a googlecolab environment, run the following code snippet (may take a few minutes):

In [ ]:
import os
os.environ["LD_PRELOAD"] = "";
os.environ["APPTAINER_BINDPATH"] = "/content"
os.environ["MPLCONFIGDIR"] = "/content/matplotlib-mpldir"
os.environ["LMOD_CMD"] = "/usr/share/lmod/lmod/libexec/lmod"

!curl -J -O https://raw.githubusercontent.com/NeuroDesk/neurocommand/main/googlecolab_setup.sh
!chmod +x googlecolab_setup.sh
!./googlecolab_setup.sh

os.environ["MODULEPATH"] = ':'.join(map(str, list(map(lambda x: os.path.join(os.path.abspath('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/'), x),os.listdir('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/')))))

Next, we must load the FSL module, allowing us to use all the FSL-functions.

FSL is a library of analysis tools for FMRI, MRI and DTI brain imaging data ([FSL website](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki)).

In [16]:
import lmod
await lmod.avail()
await lmod.load('fsl/6.0.7.1')
await lmod.list()


The following have been reloaded with a version change:
  1) fsl/6.0.4 => fsl/6.0.7.1




['fmriprep/21.0.1', 'fsl/6.0.7.1']

In [ ]:
%%bash
pip install dipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 12.3 MB/s eta 0:00:00


Next, we will download some open-source fMRI dataset from GitHub (may take a few minutes). In this case, a dataset from the OpenNeuroDatasets.

In [ ]:
%%bash
# download data
git config --global user.name "GSDrenthen"
git config --global user.email "g.s.drenthen@gmail.com"
datalad install https://github.com/OpenNeuroDatasets/ds004215.git
cd ds004215 && datalad get sub-ON05530/ses-01/func

Check the contents of the downloaded subject


In [ ]:
%%bash
echo $'Content of subject folder'
cd /content/ds004215/sub-ON05530/
tree

In [ ]:
import nibabel as nib
import numpy as np
from matplotlib import transforms
from scipy import ndimage
import pickle
import matplotlib.pyplot as plt

# load data

def view_slices_3d(image_3d, slice_nbr, vmin, vmax, title=''):
  fig = plt.figure(figsize=(15, 4))
  plt.suptitle(title, fontsize=10)

  plt.subplot(131)
  plt.imshow(np.take(image_3d[:,:,:,0], slice_nbr, 2), vmin=vmin, vmax=vmax, cmap='gray')
  plt.title('Axial');

  plt.subplot(132)
  image_rot = ndimage.rotate(np.take(image_3d[:,:,:,0], slice_nbr, 1),90)
  plt.imshow(image_rot, vmin=vmin, vmax=vmax, cmap='gray')
  plt.title('Coronal');

  plt.subplot(133)
  image_rot = ndimage.rotate(np.take(image_3d[:,:,:,0], slice_nbr, 0),90)
  plt.imshow(image_rot, vmin=vmin, vmax=vmax, cmap='gray')
  plt.title('Sagittal');
  cbar=plt.colorbar()

forward = nib.load('/content/ds004215/sub-ON05530/ses-01/func/sub-ON05530_ses-01_task-rest_dir-forward_bold.nii.gz').get_fdata()
reverse = nib.load('/content/ds004215/sub-ON05530/ses-01/func/sub-ON05530_ses-01_task-rest_dir-reverse_bold.nii.gz').get_fdata()
diffim = np.expand_dims(forward[:,:,:,0]-reverse[:,:,:,0], -1)
view_slices_3d(forward, slice_nbr=20, vmin=0, vmax=6000, title='forward fMRI scan')
view_slices_3d(reverse, slice_nbr=20, vmin=0, vmax=6000, title='reverse fMRI scan')
view_slices_3d(diffim, slice_nbr=20, vmin=0, vmax=6000, title='difference image - the highlights indicate the areas suffering from the distortions')


In [ ]:
%%bash
readouttime_Reverse=$(grep -A0 "TotalReadoutTime" /content/ds004215/sub-ON05530/ses-01/func/sub-ON05530_ses-01_task-rest_dir-forward_bold.json | grep -o "[0-9]*\.[0-9]*")
readouttime=$(grep -A0 "TotalReadoutTime" /content/ds004215/sub-ON05530/ses-01/func/sub-ON05530_ses-01_task-rest_dir-reverse_bold.json | grep -o "[0-9]*\.[0-9]*")
echo -e "-1 0 0 $readouttime\n1 0 0 $readouttime_Reverse" > acquisitionsParameters.txt

ls -a

In [ ]:
%%bash
fslroi /content/ds004215/sub-ON05530/ses-01/func/sub-ON05530_ses-01_task-rest_dir-forward_bold.nii forward.nii 0 1
fslroi /content/ds004215/sub-ON05530/ses-01/func/sub-ON05530_ses-01_task-rest_dir-reverse_bold.nii reverse.nii 0 1
fslmerge -t for_rev.nii forward.nii reverse.nii

topup \
--imain=for_rev.nii \
--datain=acquisitionsParameters.txt \
--config=b02b0.cnf \
--fout=fmap.nii \
--iout=fmrigood.nii \
--out=topup_for_rev

applytopup \
--imain=/content/ds004215/sub-ON05530/ses-01/func/sub-ON05530_ses-01_task-rest_dir-forward_bold.nii \
--inindex=1 \
--datain=acquisitionsParameters.txt \
--topup=topup_for_rev \
--method=jac --interp=spline \
--out=fmri_corr.nii

In [ ]:
good = nib.load('fmri_corr.nii.gz').get_fdata()
view_slices_3d(good, slice_nbr=20, vmin=0, vmax=6000, title='Corrected fMRI scan')
view_slices_3d(forward, slice_nbr=20, vmin=0, vmax=6000, title='Uncorrected fMRI scan')

In [ ]:
%%bash
mcflirt -in fmri_corr.nii -plots

In [ ]:
%%bash
ls -a

In [ ]:
mcgood = nib.load('fmri_corr_mcf.nii.gz').get_fdata()
view_slices_3d(good, slice_nbr=20, vmin=0, vmax=6000, title='Corrected fMRI scan')
view_slices_3d(mcgood, slice_nbr=20, vmin=0, vmax=6000, title='Motion corrected fMRI scan')

In [ ]:
plt.plot(mcgood[20,20,20,5:200])
plt.plot(good[20,20,20,5:200])

In [ ]:
mp6=np.loadtxt('/content/fmri_corr_mcf.par')
plt.plot(mp6[:,0])
plt.plot(mp6[:,1])
plt.plot(mp6[:,2])

In [ ]:
plt.plot(mp6[:,3])
plt.plot(mp6[:,4])
plt.plot(mp6[:,5])

## **Frequency analysis**
One of the most important properties of any signal is its frequency and phase distribution. More commonly, especially in the context of fMRI, we are interested in the various frequencies which comprise the measured fMRI signal.
To obtain the frequency information about a signal,  we apply the Fourier transform. Let's take a look at the differences before and after motion correction

In [ ]:
fft_good = np.fft.fft(good[20,20,20,10:])
fft_mcgood = np.fft.fft(mcgood[20,20,20,10:])

# prepare the x axis in Hz
fs = 1/3 #Hz
sample_length = fft_good.shape[0]
x_axis = np.linspace(0, fs, sample_length)

In [ ]:
# plot the "good" frequencies according to nyquist
plt.figure(figsize=(15,6))
plt.subplot(121)
plt.plot(x_axis[1:95], np.square(np.abs(fft_good[1:95])))
plt.plot(x_axis[1:95], np.square(np.abs(fft_mcgood[1:95])))

plt.xlabel("Frequency [Hz]")
plt.legend(["previous fMRI signal","motion-corrected fMRI signal"])

# plot only the spectrum for 0.01-0.10
sub_x_axis = np.where((x_axis>0.01) & (x_axis<0.1))
plt.subplot(122)
plt.plot(x_axis[sub_x_axis], np.abs(fft_good[sub_x_axis]))
plt.plot(x_axis[sub_x_axis], np.abs(fft_mcgood[sub_x_axis]))
plt.axhline(y=np.mean(np.abs(fft_good[sub_x_axis])), color='r', linestyle='--')
plt.axhline(y=np.mean(np.abs(fft_mcgood[sub_x_axis])), color='g', linestyle='--')
plt.legend(["fMRI signal",
           "motion-corrected fMRI signal",
           "mean fMRI signal",
           "mean motion-corrected fMRI signal"])

### ALFF & fALFF
One standard approach to extract information is to take a look at the ratio in power of a certain frequency band against the total signal power.

Usually, the band 0.01-0.1 is thought to reflect the neuronal activations. This band encapsulates the slow-5, slow-4 and partially slow-3 frequencies which were identified previously to be of neuronal oscillatory origin ([Gohel et al 2015](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4313418/)). The fMRI otherwise contains multiple other sources of noise, notably cardiac and respiratory.

#### fALFF
In order to reduce the effect of the physiological noise on the ALFF values, we can devide the ALFF by the total power of the entire measurable spectrum



In [ ]:
# Calculate ALFF
fmridat = mcgood[:,:,:,10:]
freq_steps = np.fft.fftfreq(fmridat.shape[-1], 1/fs) # this is the same as we calculated before for the x_axis manually

fmri2d = np.reshape(fmridat, (np.prod(fmridat.shape[0:3]), fmridat.shape[-1] ), 'F')
fft_fmri2d = np.abs(np.fft.fft(fmri2d, axis=-1)) # should be squared?

low_freq = 0.01
high_freq = 0.1

freq_mask = np.logical_and(freq_steps >= low_freq, freq_steps <= high_freq)
alff = np.mean(fft_fmri2d[:, freq_mask], axis=-1)


# fALFF
total_power = np.sum(fft_fmri2d[:, freq_steps > 0], axis=-1)
falff = np.divide(alff, total_power, out=np.zeros_like(alff), where=total_power!=0) # this instead of just alff/total_power is to avoid warnings and delays due to divisions by zero


# reshape and view
alff = np.reshape(alff, fmridat.shape[:-1])
falff = np.reshape(falff, fmridat.shape[:-1])


In [ ]:
view_slices_3d(np.expand_dims(alff, -1), slice_nbr=20, vmin=0, vmax=500, title='Corrected fMRI scan')
view_slices_3d(np.expand_dims(falff, -1), slice_nbr=20, vmin=0, vmax=0.05, title='Corrected fMRI scan')

## ReHo
One of the main concepts for analyzing the fMRI data is correlation. We would like to establish which voxels or areas correlate with each other in terms of the fMRI activity over time. If the correlation is high, we can assume that those voxels and regions are functionally connected.

Regional homoegeneity can be viewed as a special type of correlation analysis. To calculate ReHo, we take a look at the correlations between adjacent voxels. We might assume that voxels close to each other will be intrinsicely correlated, which is true (note: add a seed-based voxel connectivity). However, abnormal localized synchornicity (i.e. significant correlation of the adjacent voxels) can indicate neuronal abnormality and underlying pathology
. This has been studied in multiple diseases and shown to yield promising diagnostic results (add refs)

The standard way of calculating ReHo is to use the Kendall's coefficient of concordance which basically measures how a group of raters (in here a group of voxels) agree with each other in terms of evaluating something (in here the activity over time). Unlike standard Pearson or Spearman, Kendall ranges from 0 to 1 which is makes the interpretation easier.